In [50]:
import tensorflow as tf
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from mlxtend.preprocessing import one_hot
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

for i in range(3):
    # set CSV file list
    df = pd.read_csv('data/train.csv')
    #del df['id']

    # 카테고리를 숫자로 변환
    columnsToEncode = list(df.select_dtypes(include=['object']))
    le = LabelEncoder()
    for feature in columnsToEncode:
        df["species"] = le.fit_transform(df["species"])
    # from sklearn.model_selection import train_test_split
    
    XX = df.drop(['id','species'], axis=1)
    X_scaled = preprocessing.scale(XX)
    y = df.species

    x_data, x_test, y_data, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state = i+10)
    # x_data = df.iloc[:800,1:]
    # y_data = df.iloc[:800,0]
    # x_test = df.iloc[800:,1:]
    # y_test = df.iloc[800:,0]
    #카테고리값을 one-hot encoding
    y_data = one_hot(y_data)
    y_test = one_hot(y_test)

    x_data = np.array(x_data, dtype='float32')
    y_data = np.array(y_data, dtype='float32')
    x_test = np.array(x_test, dtype='float32')
    y_test = np.array(y_test, dtype='float32')
    tf.reset_default_graph()
    tf.set_random_seed(777)  # for reproducibility
    learning_rate = 0.0001
    training_epochs = 10000
    #batch_size = 100
    keep_prob = tf.placeholder(tf.float32)


    X = tf.placeholder(tf.float32, [None, 192])
    Y = tf.placeholder(tf.float32, [None, 99])

    W1 = tf.get_variable("W1", shape=[192, 800],
                         initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([800]))
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

    W2 = tf.get_variable("W2", shape=[800, 800],
                         initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([800]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

    W3 = tf.get_variable("W3", shape=[800, 800],
                         initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([800]))
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

#     W4 = tf.get_variable("W4", shape=[800, 800],
#                          initializer=tf.contrib.layers.xavier_initializer())
#     b4 = tf.Variable(tf.random_normal([800]))
#     L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
#     L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

#     W5 = tf.get_variable("W5", shape=[800, 800],
#                          initializer=tf.contrib.layers.xavier_initializer())
#     b5 = tf.Variable(tf.random_normal([800]))
#     L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)
#     L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

    # W6 = tf.get_variable("W6", shape=[800, 800],
    #                      initializer=tf.contrib.layers.xavier_initializer())
    # b6 = tf.Variable(tf.random_normal([800]))
    # L6 = tf.nn.relu(tf.matmul(L5, W6) + b6)
    # L6 = tf.nn.dropout(L6, keep_prob=keep_prob)

    W6 = tf.get_variable("W6", shape=[800, 99],
                         initializer=tf.contrib.layers.xavier_initializer())
    b6 = tf.Variable(tf.random_normal([99]))
    hypothesis = tf.matmul(L3, W6) + b6

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=hypothesis, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # train my model
    for epoch in range(training_epochs):
        feed_dict = {X: x_data, Y: y_data, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        #if epoch % 50 == 0:
        print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(c))

    print('Learning Finished!')

    # Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy:', sess.run(accuracy, feed_dict={
          X: x_test, Y: y_test, keep_prob: 1}))
    score.append(sess.run(accuracy, feed_dict={
          X: x_test, Y: y_test, keep_prob: 1}))

    # # Get one and predict
    # r = random.randint(0, mnist.test.num_examples - 1)
    # print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
    # print("Prediction: ", sess.run(
    #     tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))


Epoch: 0000 cost = 6.481434822
Epoch: 0001 cost = 6.206984043
Epoch: 0002 cost = 5.992747784
Epoch: 0003 cost = 5.778319836
Epoch: 0004 cost = 5.676650524
Epoch: 0005 cost = 5.532575607
Epoch: 0006 cost = 5.359845638
Epoch: 0007 cost = 5.233693123
Epoch: 0008 cost = 5.343541145
Epoch: 0009 cost = 5.169762611
Epoch: 0010 cost = 5.136382103
Epoch: 0011 cost = 5.009916782
Epoch: 0012 cost = 4.977970123
Epoch: 0013 cost = 4.887985706
Epoch: 0014 cost = 4.823018551
Epoch: 0015 cost = 4.839081287
Epoch: 0016 cost = 4.772271156
Epoch: 0017 cost = 4.749011993
Epoch: 0018 cost = 4.714751720
Epoch: 0019 cost = 4.676244259
Epoch: 0020 cost = 4.566072941
Epoch: 0021 cost = 4.497436523
Epoch: 0022 cost = 4.503049850
Epoch: 0023 cost = 4.463594913
Epoch: 0024 cost = 4.439726353
Epoch: 0025 cost = 4.386768341
Epoch: 0026 cost = 4.333093643
Epoch: 0027 cost = 4.306559086
Epoch: 0028 cost = 4.261602879
Epoch: 0029 cost = 4.285722733
Epoch: 0030 cost = 4.208400726
Epoch: 0031 cost = 4.152257919
Epoch: 0

Epoch: 0265 cost = 0.116600618
Epoch: 0266 cost = 0.115891896
Epoch: 0267 cost = 0.115508325
Epoch: 0268 cost = 0.112032972
Epoch: 0269 cost = 0.118160486
Epoch: 0270 cost = 0.117791303
Epoch: 0271 cost = 0.116105549
Epoch: 0272 cost = 0.121485531
Epoch: 0273 cost = 0.116786577
Epoch: 0274 cost = 0.159557864
Epoch: 0275 cost = 0.107161693
Epoch: 0276 cost = 0.113977574
Epoch: 0277 cost = 0.118236266
Epoch: 0278 cost = 0.114321060
Epoch: 0279 cost = 0.107136540
Epoch: 0280 cost = 0.115809657
Epoch: 0281 cost = 0.100823984
Epoch: 0282 cost = 0.118268274
Epoch: 0283 cost = 0.100406513
Epoch: 0284 cost = 0.110841371
Epoch: 0285 cost = 0.111977652
Epoch: 0286 cost = 0.107746102
Epoch: 0287 cost = 0.105451778
Epoch: 0288 cost = 0.111421317
Epoch: 0289 cost = 0.100971766
Epoch: 0290 cost = 0.095153078
Epoch: 0291 cost = 0.096338190
Epoch: 0292 cost = 0.100784808
Epoch: 0293 cost = 0.107784584
Epoch: 0294 cost = 0.098956071
Epoch: 0295 cost = 0.090557702
Epoch: 0296 cost = 0.098457731
Epoch: 0

Epoch: 0530 cost = 0.028594252
Epoch: 0531 cost = 0.035703827
Epoch: 0532 cost = 0.023574606
Epoch: 0533 cost = 0.023325600
Epoch: 0534 cost = 0.025452089
Epoch: 0535 cost = 0.026082372
Epoch: 0536 cost = 0.024206419
Epoch: 0537 cost = 0.028314177
Epoch: 0538 cost = 0.027856385
Epoch: 0539 cost = 0.025571484
Epoch: 0540 cost = 0.022064615
Epoch: 0541 cost = 0.029028444
Epoch: 0542 cost = 0.023931505
Epoch: 0543 cost = 0.024657175
Epoch: 0544 cost = 0.029780092
Epoch: 0545 cost = 0.032374132
Epoch: 0546 cost = 0.028310111
Epoch: 0547 cost = 0.023567703
Epoch: 0548 cost = 0.019666344
Epoch: 0549 cost = 0.021553300
Epoch: 0550 cost = 0.021726087
Epoch: 0551 cost = 0.023162987
Epoch: 0552 cost = 0.026718060
Epoch: 0553 cost = 0.030276211
Epoch: 0554 cost = 0.029721405
Epoch: 0555 cost = 0.022420451
Epoch: 0556 cost = 0.020255240
Epoch: 0557 cost = 0.023134766
Epoch: 0558 cost = 0.022994980
Epoch: 0559 cost = 0.026363762
Epoch: 0560 cost = 0.020173304
Epoch: 0561 cost = 0.020028263
Epoch: 0

Epoch: 0795 cost = 0.012425106
Epoch: 0796 cost = 0.011784922
Epoch: 0797 cost = 0.009722744
Epoch: 0798 cost = 0.013683945
Epoch: 0799 cost = 0.010509332
Epoch: 0800 cost = 0.011080434
Epoch: 0801 cost = 0.010535852
Epoch: 0802 cost = 0.021834513
Epoch: 0803 cost = 0.013209691
Epoch: 0804 cost = 0.016392786
Epoch: 0805 cost = 0.009626416
Epoch: 0806 cost = 0.013493538
Epoch: 0807 cost = 0.008671576
Epoch: 0808 cost = 0.006632990
Epoch: 0809 cost = 0.009937334
Epoch: 0810 cost = 0.010042335
Epoch: 0811 cost = 0.009686155
Epoch: 0812 cost = 0.011617827
Epoch: 0813 cost = 0.007648522
Epoch: 0814 cost = 0.014500505
Epoch: 0815 cost = 0.010475037
Epoch: 0816 cost = 0.011582786
Epoch: 0817 cost = 0.013129513
Epoch: 0818 cost = 0.010627038
Epoch: 0819 cost = 0.009338795
Epoch: 0820 cost = 0.008430517
Epoch: 0821 cost = 0.011088659
Epoch: 0822 cost = 0.015537417
Epoch: 0823 cost = 0.009834289
Epoch: 0824 cost = 0.017853696
Epoch: 0825 cost = 0.011141746
Epoch: 0826 cost = 0.013977137
Epoch: 0

Epoch: 1060 cost = 0.007112850
Epoch: 1061 cost = 0.006188754
Epoch: 1062 cost = 0.004383755
Epoch: 1063 cost = 0.006826361
Epoch: 1064 cost = 0.007837174
Epoch: 1065 cost = 0.007641079
Epoch: 1066 cost = 0.005989634
Epoch: 1067 cost = 0.008879086
Epoch: 1068 cost = 0.004372336
Epoch: 1069 cost = 0.005779438
Epoch: 1070 cost = 0.005738198
Epoch: 1071 cost = 0.006797876
Epoch: 1072 cost = 0.009503898
Epoch: 1073 cost = 0.006246206
Epoch: 1074 cost = 0.006171471
Epoch: 1075 cost = 0.013883166
Epoch: 1076 cost = 0.008501804
Epoch: 1077 cost = 0.011267487
Epoch: 1078 cost = 0.006171512
Epoch: 1079 cost = 0.007786334
Epoch: 1080 cost = 0.008835768
Epoch: 1081 cost = 0.006697268
Epoch: 1082 cost = 0.006711844
Epoch: 1083 cost = 0.004774807
Epoch: 1084 cost = 0.008064650
Epoch: 1085 cost = 0.007014431
Epoch: 1086 cost = 0.006942423
Epoch: 1087 cost = 0.006540061
Epoch: 1088 cost = 0.005013837
Epoch: 1089 cost = 0.005727853
Epoch: 1090 cost = 0.005992721
Epoch: 1091 cost = 0.005689652
Epoch: 1

...

Epoch: 8883 cost = 0.000030072
Epoch: 8884 cost = 0.000172757
Epoch: 8885 cost = 0.000105363
Epoch: 8886 cost = 0.000021471
Epoch: 8887 cost = 0.000189704
Epoch: 8888 cost = 0.000013171
Epoch: 8889 cost = 0.000076085
Epoch: 8890 cost = 0.000047987
Epoch: 8891 cost = 0.000023896
Epoch: 8892 cost = 0.000031497
Epoch: 8893 cost = 0.000045536
Epoch: 8894 cost = 0.000024564
Epoch: 8895 cost = 0.000051135
Epoch: 8896 cost = 0.000020378
Epoch: 8897 cost = 0.000052008
Epoch: 8898 cost = 0.000004856
Epoch: 8899 cost = 0.000037274
Epoch: 8900 cost = 0.000021922
Epoch: 8901 cost = 0.000021462
Epoch: 8902 cost = 0.000009883
Epoch: 8903 cost = 0.000178165
Epoch: 8904 cost = 0.000077098
Epoch: 8905 cost = 0.000009633
Epoch: 8906 cost = 0.000018738
Epoch: 8907 cost = 0.000022400
Epoch: 8908 cost = 0.000023384
Epoch: 8909 cost = 0.000052009
Epoch: 8910 cost = 0.000015027
Epoch: 8911 cost = 0.000019898
Epoch: 8912 cost = 0.000006466
Epoch: 8913 cost = 0.000041170
Epoch: 8914 cost = 0.000056081
Epoch: 8

Epoch: 9148 cost = 0.000031549
Epoch: 9149 cost = 0.000887130
Epoch: 9150 cost = 0.000018777
Epoch: 9151 cost = 0.000038527
Epoch: 9152 cost = 0.000061478
Epoch: 9153 cost = 0.000031159
Epoch: 9154 cost = 0.000034309
Epoch: 9155 cost = 0.000056686
Epoch: 9156 cost = 0.000031101
Epoch: 9157 cost = 0.000025336
Epoch: 9158 cost = 0.000044112
Epoch: 9159 cost = 0.000088177
Epoch: 9160 cost = 0.000132128
Epoch: 9161 cost = 0.000223238
Epoch: 9162 cost = 0.000022940
Epoch: 9163 cost = 0.000024953
Epoch: 9164 cost = 0.000049950
Epoch: 9165 cost = 0.000034896
Epoch: 9166 cost = 0.000006777
Epoch: 9167 cost = 0.000016016
Epoch: 9168 cost = 0.000027392
Epoch: 9169 cost = 0.000011945
Epoch: 9170 cost = 0.000026727
Epoch: 9171 cost = 0.000014641
Epoch: 9172 cost = 0.000017883
Epoch: 9173 cost = 0.000012108
Epoch: 9174 cost = 0.000020873
Epoch: 9175 cost = 0.000019699
Epoch: 9176 cost = 0.000023225
Epoch: 9177 cost = 0.000037870
Epoch: 9178 cost = 0.000057604
Epoch: 9179 cost = 0.000025805
Epoch: 9

Epoch: 9413 cost = 0.000119252
Epoch: 9414 cost = 0.000046007
Epoch: 9415 cost = 0.000012545
Epoch: 9416 cost = 0.000012009
Epoch: 9417 cost = 0.000010009
Epoch: 9418 cost = 0.000024050
Epoch: 9419 cost = 0.000029069
Epoch: 9420 cost = 0.000187144
Epoch: 9421 cost = 0.000041460
Epoch: 9422 cost = 0.000011876
Epoch: 9423 cost = 0.000011945
Epoch: 9424 cost = 0.000047563
Epoch: 9425 cost = 0.000297668
Epoch: 9426 cost = 0.003853907
Epoch: 9427 cost = 0.000018680
Epoch: 9428 cost = 0.000029970
Epoch: 9429 cost = 0.000013266
Epoch: 9430 cost = 0.000080428
Epoch: 9431 cost = 0.000024651
Epoch: 9432 cost = 0.000103109
Epoch: 9433 cost = 0.000029071
Epoch: 9434 cost = 0.000022165
Epoch: 9435 cost = 0.001413234
Epoch: 9436 cost = 0.000272172
Epoch: 9437 cost = 0.000012371
Epoch: 9438 cost = 0.000017257
Epoch: 9439 cost = 0.000036201
Epoch: 9440 cost = 0.002244542
Epoch: 9441 cost = 0.000011819
Epoch: 9442 cost = 0.000034667
Epoch: 9443 cost = 0.000009240
Epoch: 9444 cost = 0.000020450
Epoch: 9

Epoch: 9678 cost = 0.000069835
Epoch: 9679 cost = 0.000034412
Epoch: 9680 cost = 0.000065444
Epoch: 9681 cost = 0.000023410
Epoch: 9682 cost = 0.000020411
Epoch: 9683 cost = 0.000058558
Epoch: 9684 cost = 0.000172490
Epoch: 9685 cost = 0.000052547
Epoch: 9686 cost = 0.000092171
Epoch: 9687 cost = 0.000128251
Epoch: 9688 cost = 0.000018430
Epoch: 9689 cost = 0.000014485
Epoch: 9690 cost = 0.000073789
Epoch: 9691 cost = 0.000017099
Epoch: 9692 cost = 0.000012514
Epoch: 9693 cost = 0.000015519
Epoch: 9694 cost = 0.000021999
Epoch: 9695 cost = 0.000019215
Epoch: 9696 cost = 0.000118019
Epoch: 9697 cost = 0.000124825
Epoch: 9698 cost = 0.000026191
Epoch: 9699 cost = 0.000089499
Epoch: 9700 cost = 0.000041671
Epoch: 9701 cost = 0.000008965
Epoch: 9702 cost = 0.000041302
Epoch: 9703 cost = 0.000017248
Epoch: 9704 cost = 0.000042068
Epoch: 9705 cost = 0.000341211
Epoch: 9706 cost = 0.000017879
Epoch: 9707 cost = 0.000037691
Epoch: 9708 cost = 0.000023783
Epoch: 9709 cost = 0.000075308
Epoch: 9

Epoch: 9943 cost = 0.000041665
Epoch: 9944 cost = 0.000025521
Epoch: 9945 cost = 0.000046912
Epoch: 9946 cost = 0.000016241
Epoch: 9947 cost = 0.000014939
Epoch: 9948 cost = 0.000018849
Epoch: 9949 cost = 0.000022510
Epoch: 9950 cost = 0.000125116
Epoch: 9951 cost = 0.000010748
Epoch: 9952 cost = 0.000005299
Epoch: 9953 cost = 0.000053901
Epoch: 9954 cost = 0.000013378
Epoch: 9955 cost = 0.000007650
Epoch: 9956 cost = 0.000034009
Epoch: 9957 cost = 0.000012728
Epoch: 9958 cost = 0.000045304
Epoch: 9959 cost = 0.000011749
Epoch: 9960 cost = 0.000024311
Epoch: 9961 cost = 0.000008057
Epoch: 9962 cost = 0.000044561
Epoch: 9963 cost = 0.000028077
Epoch: 9964 cost = 0.000080415
Epoch: 9965 cost = 0.000013029
Epoch: 9966 cost = 0.000007617
Epoch: 9967 cost = 0.000015346
Epoch: 9968 cost = 0.000004972
Epoch: 9969 cost = 0.000011697
Epoch: 9970 cost = 0.000007273
Epoch: 9971 cost = 0.000017625
Epoch: 9972 cost = 0.000010992
Epoch: 9973 cost = 0.000006894
Epoch: 9974 cost = 0.000015337
Epoch: 9

In [40]:
score = []

In [46]:
score

[0.81818181, 0.89898992, 0.92929292, 0.89898992]

In [49]:
x_data

array([[-0.88257545, -0.73488045,  1.25669408, ..., -0.49497402,
         0.97732466,  0.56279892],
       [-0.68459308, -0.73488045,  0.50066841, ..., -0.41990992,
        -0.65214312,  0.04781805],
       [ 0.60239357,  0.82419175, -0.40659332, ..., -0.3448202 ,
         0.05634638, -0.85338742],
       ..., 
       [-0.78358424, -0.58401251, -0.48219201, ..., -0.29476893,
         2.1108644 ,  2.19352031],
       [-0.68459308, -0.1816721 , -0.78458679, ..., -0.44492275,
         0.05634638,  1.29231477],
       [-0.58560193, -0.33254009,  3.52492571, ..., -0.36985865,
        -0.65214312, -0.68174177]], dtype=float32)